## Data Staging

In [52]:
import pandas as pd
from datetime import datetime
import praw

In [53]:
# Reddit API credentials
reddit = praw.Reddit(
    client_id="CTY-7JaWjWmegP8ZSJcmnw",      # Your Client ID
    client_secret="8q3ftj_Lpi3p7Z8-DlRM8p4bTOSsVg",  # Your Client Secret
    user_agent="USAID_Scraper"  # Custom identifier
)

In [54]:
usaid_queries = [
    "USAID"
]

regions = [
    "Africa", "Middle East", "Asia", "Europe", "Latin America", "South America"
]

In [ ]:
results = []

# Fetch posts and comments
for query in usaid_queries:
    for region in regions:
        full_query = f"{query} {region}"
        print(f"\n🔹 Searching for: {full_query}")

        for post in reddit.subreddit("all").search(full_query, limit=500, sort="new"):
            # Get top 5 comments
            post.comments.replace_more(limit=0)  # Load all comments
            top_comments = [comment.body for comment in post.comments.list()[:5]]
            created_at_utc = post.created_utc
            created_at_human = datetime.utcfromtimestamp(created_at_utc).strftime('%Y-%m-%d %H:%M:%S')
            post.author.name if post.author else "Unknown" 

            post_data = {
                "Post ID": post.id,
                "region": region,
                "Text": post.selftext,
                "title": post.title,
                "Author":post.author.name,
                "subreddit": post.subreddit.display_name,
                "score": post.score,
                "Upvote Ratio": post.upvote_ratio,
                "Number of Comments": post.num_comments,
                "Created At": created_at_utc,
                "url": post.url,
                "Created Timestamp": created_at_human,
                "comments": " || ".join(top_comments)  
            }
            results.append(post_data)  
#df = pd.DataFrame(results)
df.to_csv("usaid_reddit_posts_New.csv", index=False)


🔹 Searching for: USAID Africa

🔹 Searching for: USAID Middle East

🔹 Searching for: USAID Asia

🔹 Searching for: USAID Europe

🔹 Searching for: USAID Latin America

🔹 Searching for: USAID South America


## Sentiment Analysis

In [55]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download the required NLTK data
nltk.download('stopwords')  # For the stopwords
nltk.download('punkt')      # For word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perpe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\perpe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
import pandas as pd

# Try specifying a different encoding
df = pd.read_csv("usaid_reddit_posts_New.csv", encoding='latin1')  # You can also try 'ISO-8859-1' or 'cp1252'

# If it works, the dataframe will load without errors
df.head()

,Post ID,region,Text,title,Author,subreddit,score,Upvote Ratio,Number of Comments,Created At,url,Created Timestamp,comments,Unnamed: 13
0,1iis138,Africa,NaN,The Real Reason Musk is After USAID: They Help...,shake1010,MarchAgainstNazis,860,0.99,21,1738808995,https://i.redd.it/arjpjejvjfhe1.jpeg,2/6/2025 2:29,Welcome to /r/MarchAgainstNazis! \n\n**Please...,NaN
1,1iiontm,Africa,NaN,Usaid Delivering Foreign Aid to starving child...,trailer8k,Trailerclub,1,1,0,1738799255,https://v.redd.it/w5xmfqsvqehe1,2/5/2025 23:47,NaN,NaN
2,1iinl7c,Africa,How USAid has been spending taxpayer money\n\n...,How has USAID been spending taxpayer money?,ProtectedHologram,conspiracy,38,0.75,33,1738796394,https://www.reddit.com/r/conspiracy/comments/1...,2/5/2025 22:59,###[Meta] Sticky Comment\n\n[Rule 2](https://w...,NaN
3,1iini2u,Africa,NaN,Here's my list of exposed corrupt money launde...,Believe_143,LibTears,183,0.99,23,1738796165,https://i.redd.it/qt4m216qhehe1.jpeg,2/5/2025 22:56,And yet...hurricane victims in NC and fire vic...,NaN
4,1iijyzc,Africa,Source: https://x.com/kenklippenstein/status/1...,Some people say that USAIDâs aid is exclusiv...,SnooAdvice725,Lebanese,123,0.98,12,1738787376,https://i.redd.it/esuop84lrdhe1.jpeg,2/5/2025 20:29,So they're interfering in international politi...,NaN


In [57]:
import nltk
import os

# Specify the directory for NLTK data
nltk_data_dir = os.path.expanduser("~/nltk_data")
nltk.data.path.append(nltk_data_dir)

# Download necessary NLTK data
nltk.download('stopwords', download_dir=nltk_data_dir)
nltk.download


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perpe/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


<bound method Downloader.download of <nltk.downloader.Downloader object at 0x0000027745F18F10>>

In [58]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perpe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\perpe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [59]:
# Define a cleaning function
def clean_text(text):
    if isinstance(text, str):  # Only process strings
        text = text.lower()  # Convert to lowercase
        text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)  # Remove URLs
        text = re.sub(r'\W', ' ', text)  # Remove special characters and punctuation
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        tokens = word_tokenize(text)  # Tokenize the text
        filtered_words = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
        return ' '.join(filtered_words)
    else:
        return ""  # Return an empty string for non-string values

# Apply cleaning to the 'Text' column using .loc
df.loc[:, 'Cleaned_Text'] = df['Text'].apply(clean_text)

# Save the cleaned dataset
df.to_csv("cleaned_usaid_data.csv", index=False)
print("Data cleaned and saved.")


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\perpe/nltk_data'
    - 'c:\\Users\\perpe\\anaconda3\\nltk_data'
    - 'c:\\Users\\perpe\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\perpe\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\perpe\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\perpe/nltk_data'
    - 'C:\\Users\\perpe/nltk_data'
    - 'C:\\Users\\perpe/nltk_data'
    - 'C:\\Users\\perpe/nltk_data'
    - 'C:/Users/perpe/nltk_data'
    - 'C:\\Users\\perpe/nltk_data'
    - 'C:\\Users\\perpe/nltk_data'
    - 'C:\\Users\\perpe\\AppData\\Roaming\\nltk_data'
    - 'C:\\Us

In [63]:
from textblob import TextBlob

In [60]:
df = pd.read_csv("cleaned_usaid_data.csv")

In [64]:
# Ensure 'Cleaned_Text' column contains only strings
df['Cleaned_Text'] = df['Cleaned_Text'].astype(str)

# Sentiment analysis function
def get_sentiment(text):
    if not text or text.isspace():  # Check for empty or whitespace-only strings
        return "Neutral"
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return "Positive"
    elif analysis.sentiment.polarity < 0:
        return "Negative"
    else:
        return "Neutral"

# Apply sentiment analysis to the cleaned text
df['Sentiment'] = df['Cleaned_Text'].apply(get_sentiment)


In [65]:
# Save the dataset with sentiment results
df.to_csv("sentiment_usaid_data.csv", index=False)

print("Sentiment analysis completed and data saved.")

Sentiment analysis completed and data saved.


In [15]:
print(df.columns)


Index(['Post ID', 'region', 'Text', 'title', 'Author', 'subreddit', 'score',
       'Upvote Ratio', 'Number of Comments', 'Created At', 'url',
       'Created Timestamp', 'comments'],
      dtype='object')


In [ ]:
def chatbot():
    print("Hello! I'm your USAID information assistant. Ask me anything about USAID.")
    
    while True:
        user_input = input("You: ").lower()
        
        if 'usaid' in user_input:
            # Query your dataset for relevant information
            relevant_data = data[data['Text'].str.contains('usaid', case=False, na=False)]
            
            if not relevant_data.empty:
                print("Bot: Here's some information about USAID:")
                for index, row in relevant_data.head().iterrows():
                    print(f"- {row['title']}: {row['Text']}")
            else:
                print("Bot: Sorry, I couldn't find any information about USAID.")
        else:
            print("Bot: Sorry, that's beyond my current scope. Let’s talk about something else.")
        
        # Exit condition
        if user_input in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break

# Run the chatbot
chatbot()

Hello! I'm your USAID information assistant. Ask me anything about USAID.
Bot: Here's some information about USAID:
- Will US Politics hamper worldwide missionary efforts?: US pollitics are certainly chaotic right now. My suspicion is that the MAGA brand and bashing of non-US peoples and countries will influence foreign perception of US-centered religions, particularly ones from red states. The anti-US rhetoric has escalated from last time around. My bet is that in the next number of years, Mormon recruitment efforts internationally will see declines. Big declines.

Is you are Hispanic from central and south America, would you join a church from a state that supports the deportation of your people? If you are African, would you join a church whose people voted for a person that calls your country a "shithole" or canceled the support from USAid or thought that anti-apartheid efforts in South Africa were a step backward?

From tariff threats, to promoting permanent displacement of Gazaan

In [4]:
import pandas as pd
df = pd.read_csv("sentiment_usaid_data.csv")



In [5]:
def chatbot():
    print("Hello! I'm your USAID information assistant. Ask me anything about USAID.")
    
    while True:
        user_input = input("You: ").lower()
        
        if 'usaid' in user_input:
            # Query your dataset for relevant information
            relevant_data = df[df['Text'].str.contains('usaid', case=False, na=False)]
            
            if not relevant_data.empty:
                print("Bot: Here's some information about USAID:")
                for index, row in relevant_data.head().iterrows():
                    # Extract and summarize key points about USAID
                    text = row['Text']
                    if len(text) > 200:  # Summarize long text
                        text = text[:200] + "..."
                    print(f"- {row['title']}: {text}")
            else:
                print("Bot: Sorry, I couldn't find any information about USAID.")
        else:
            print("Bot: Sorry, that's beyond my current scope. Let’s talk about something else.")
        
        # Exit condition
        if user_input in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break

# Run the chatbot
chatbot()

Hello! I'm your USAID information assistant. Ask me anything about USAID.
Bot: Sorry, that's beyond my current scope. Let’s talk about something else.
Bot: Here's some information about USAID:
- Will US Politics hamper worldwide missionary efforts?: US pollitics are certainly chaotic right now. My suspicion is that the MAGA brand and bashing of non-US peoples and countries will influence foreign perception of US-centered religions, particularly o...
- USAID has been completely unaccountable for decades,: From: [https://x.com/rapidresponse47/status/1887151291895267553](https://x.com/rapidresponse47/status/1887151291895267553)

USAID has been completely unaccountable for decades, run by bureaucrats with...
- I don't think Elon understands how running a country like a company is so bad for everyone.: Elon grew up wealthy and in South Africa which, has a lot of extreme racism against white people. From what I understand, it's pretty bad and difficult to describe how bad. That was his life 

In [6]:
import requests
import pandas as pd

# Replace with your Deepseek API key
DEEPSEEK_API_KEY = "sk-183372fafc184722ba529f2285c3539b"
DEEPSEEK_SUMMARIZE_URL = "https://api.deepseek.com/v1/summarize"  # Replace with the actual API endpoint

In [7]:
# Load your dataset
data = pd.read_csv('sentiment_usaid_data.csv')

# Function to summarize text using Deepseek API
def summarize_with_deepseek(text):
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "text": text,
        "max_length": 100  # Adjust the summary length as needed
    }
    response = requests.post(DEEPSEEK_SUMMARIZE_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("summary", "No summary available.")
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return text[:100] + "..."  # Fallback to truncation if API fails

# Chatbot function
def chatbot():
    print("Hello! I'm your USAID information assistant. Ask me anything about USAID.")
    
    while True:
        user_input = input("You: ").lower()
        
        if 'usaid' in user_input:
            # Query your dataset for relevant information
            relevant_data = data[data['Text'].str.contains('usaid', case=False, na=False)]
            
            if not relevant_data.empty:
                print("Bot: Here's some information about USAID:")
                for index, row in relevant_data.head().iterrows():
                    # Summarize the text using Deepseek API
                    summary = summarize_with_deepseek(row['Text'])
                    print(f"- {row['title']}: {summary}")
            else:
                print("Bot: Sorry, I couldn't find any information about USAID.")
        else:
            print("Bot: Sorry, that's beyond my current scope. Let’s talk about something else.")
        
        # Exit condition
        if user_input in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break

# Run the chatbot
chatbot()

Hello! I'm your USAID information assistant. Ask me anything about USAID.
Bot: Here's some information about USAID:
Error: 404 - {
  "error_msg": "Not Found. Please check the configuration."
}
- Will US Politics hamper worldwide missionary efforts?: US pollitics are certainly chaotic right now. My suspicion is that the MAGA brand and bashing of non...
Error: 404 - {
  "error_msg": "Not Found. Please check the configuration."
}
- USAID has been completely unaccountable for decades,: From: [https://x.com/rapidresponse47/status/1887151291895267553](https://x.com/rapidresponse47/statu...
Error: 404 - {
  "error_msg": "Not Found. Please check the configuration."
}
- I don't think Elon understands how running a country like a company is so bad for everyone.: Elon grew up wealthy and in South Africa which, has a lot of extreme racism against white people. Fr...
Error: 404 - {
  "error_msg": "Not Found. Please check the configuration."
}
- Please share this on all social! What USAID would do w

In [9]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key="sk-183372fafc184722ba529f2285c3539b", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

APIStatusError: Error code: 402 - {'error': {'message': 'Insufficient Balance', 'type': 'unknown_error', 'param': None, 'code': 'invalid_request_error'}}